# Exercise 3
Phase transition in 2D Ising model

In [4]:
import numpy as np
from numpy.random import randint, rand
import matplotlib.pyplot as plt


In [12]:
def initial_config(N):   
    ''' 
    Generates a random spin configuration for initial condition
    '''
    config = 2*np.random.randint(2, size=(N,N))-1
    return config

def cal_Energy(config):
    Energy = 0
    N = config.shape[0]
    for i in range(N-1):
        for j in range(N-1):
            spin_val = config[i,j]
            Energy_of_spin = spin_val * (config[i,j+1] + config[i, j-1] + config[i+1,j] + config[i-1,j])
            Energy += Energy_of_spin
        # on the edge we need to do it by hand
        spin_val = config[i,N-1]
        Energy_of_spin = spin_val * (config[i,0] + config[i, N-2] + config[i+1,N-1] + config[i-1,N-1])
        Energy += Energy_of_spin
        
    for j in range(N-1):
        spin_val = config[N-1,j]
        Energy_of_spin = spin_val * (config[N-1,j+1] + config[N-1, j-1] + config[0,j] + config[N-2,j])
        Energy += Energy_of_spin
    # on the edge we need to do it by hand
    spin_val = config[N-1,N-1]
    Energy_of_spin = spin_val * (config[N-1,0] + config[N-1, N-2] + config[0,N-1] + config[N-2,N-1])
    Energy += Energy_of_spin
    return Energy / -2 # divide here because every interaction has been counted twice


def cal_Magnetization(config):
    '''
    Magnetization of a given configuration
    '''
    mag = np.sum(config)
    return mag/config.size    

def time_step(config, T):
    i = randint(0,N)
    j = randint(0,N)
    
    Energy_before = cal_Energy(config)
    config[i,j] *= -1 # simulate flip
    Energy_after = cal_Energy(config)
    config[i,j] *= -1 # flip it back
    
    beta = 1 / T
    delta_E = Energy_after - Energy_before
    p = np.exp(-1 * delta_E * beta)
    random_number = rand()
    if (random_number <= p):
        config[i,j] = -1 * config[i,j] # accepted => flip spin


# complete sweep so that on average each spin has been offered a flip
def sweep(config, T):
    N = config.shape[0]
    for _ in range(N*N):
        time_step(config, T)
        
def many_sweeps(config, T, N=100):
    for _ in range(N):
        sweep(config, T)
        
def measurement_magnetiziation(T,N, single_measurements=5):
    config = initial_config(N)
    many_sweeps(config,T)
    results = []
    for _ in range(single_measurements):
        results.append(np.abs(cal_Magnetization(config)))
        many_sweeps(config,T,N=5)
    return np.average(results), np.std(results)

def measurement_energy(T,N, single_measurements=5):
    config = initial_config(N)
    many_sweeps(config,T)
    results = []
    for _ in range(single_measurements):
        results.append(cal_Energy(config))
        many_sweeps(config,T,N=5)
    return np.average(results), np.std(results)

def print_grid(config):
    N = config.shape[0]
    for i in range(N):
        for j in range(N):
            if (config[i,j] == 1):
                print("#", end ="")
            else:
                print(" ", end="")
        print("", end="\n")

In [21]:
def observables(L,T):
    M_ave = []
    M_std = []
    E_ave = []
    E_std = []
    for tempe in T:
        ave, std = measurement_magnetiziation(tempe,L)
        M_ave.append(ave)
        M_std.append(std)
        ave, std = measurement_energy(tempe,L)
        E_ave.append(ave / L**2)
        E_std.append(std / L**2)
    return M_ave, M_std, E_ave, E_std

In [ ]:
N = 10
T = np.linspace(.05, 10,40)

M_ave_10, M_std_10, E_ave_10, E_std_10 = observables(N, T)
M_ave_20, M_std_20, E_ave_20, E_std_20 = observables(2 * N, T)
M_ave_30, M_std_30, E_ave_30, E_std_30 = observables(3 * N, T)

In [ ]:
plt.errorbar(T, M_ave_10,xerr=.1, yerr=M_std_10, ls='',
            label='L = 10')
plt.errorbar(T, M_ave_20,xerr=.1, yerr=M_std_20, ls='',
            label='L = 20')
plt.errorbar(T, M_ave_30,xerr=.1, yerr=M_std_30, ls='',
            label='L = 30')
plt.xlabel('T')
plt.ylabel('M')

In [ ]:
plt.errorbar(T, E_ave_10,xerr=.1, yerr=E_std_10, ls='',
            label='L = 10')
plt.errorbar(T, E_ave_20,xerr=.1, yerr=E_std_20, ls='',
            label='L = 20')
plt.errorbar(T, E_ave_30,xerr=.1, yerr=E_std_30, ls='',
            label='L = 30')plt.xlabel('T')
plt.ylabel('E')
plt.xlabel('T')